In [1]:
from h2_molecule import *
Precision.PRECISION = 5
Precision.update_threshold()

import pandas as pd
import seaborn as sns

In [2]:
problem_instance = H2MoleculeInstance({0:0}, H2ExperimentID.P0_CliffordT)
initial_state = problem_instance.initial_state[0].to_np_array()

In [17]:
H = get_hamiltonian(H2ExperimentID.P0_CliffordT)

In [27]:
H.to_matrix()

array([[-1.82172106+0.j,  0.18177154+0.j],
       [ 0.18177154+0.j, -0.26673071+0.j]])

In [26]:
times = []
energies = []
for t in range(0, 14):
    times.append(t)
    state_t = schroedinger_equation(H, complex(0, -t), initial_state)
    energies.append(get_energy(H, state_t))
    
state_t

array([ 0.99341393+0.j, -0.11458085+0.j])

In [20]:
energies

[-1.8217210634986007,
 -1.8418158349798708,
 -1.8426509463505343,
 -1.8426852162182255,
 -1.8426866217920521,
 -1.8426866794402308,
 -1.8426866818046106,
 -1.8426866819015824,
 -1.8426866819055598,
 -1.8426866819057228,
 -1.84268668190573,
 -1.8426866819057306,
 -1.8426866819057304,
 -1.84268668190573]

In [28]:
eigvals, eigvecs = np.linalg.eig(H)


array([-1.84268668+0.j, -0.24576509+0.j])

In [29]:
eigvecs

array([[ 0.99341393+0.j,  0.11458085+0.j],
       [-0.11458085-0.j,  0.99341393+0.j]])

In [21]:
eigenvalues, eigenvectors = np.linalg.eig(H)

In [23]:
eigenvalues

array([-1.84268668+0.j, -0.24576509+0.j])

In [22]:
eigenvectors

array([[ 0.99341393+0.j,  0.11458085+0.j],
       [-0.11458085-0.j,  0.99341393+0.j]])

In [11]:
state_t

array([ 0.99341393+0.j, -0.11458085+0.j])

In [25]:
get_energy(H, state_t)

-1.84268668190573

In [24]:
get_fidelity(eigenvectors[0], state_t)

0.9481743732006236